In [81]:
import csv
import time
from itertools import combinations

In [82]:
# initialize
item_list = {}
global_list = {}
global_rule_list = []
data = []

In [83]:
# constants
MIN_SUPPORT = 300
MIN_CONF = 0.3

In [84]:
#functions
def support_check(item_list:dict):
    # check if items have more than minimum support else remove them
    update = {k:v for k,v in item_list.items() if v >= MIN_SUPPORT}
    global_list.update(update)
    return update


# given an item list of level l generate items of level l+1
def next_gen(item_list:dict):
    
    ck = {}
    items = item_list.keys()
    for pair in combinations(items,2):
        a = pair[0]
        b = pair[1]
        # if they differ by one
        if len(a.difference(b)) == 1:
            ck[a.union(b)] = 0

    return ck


def get_count(item_list:dict,data:list):
    # get counts of  each item list
    for item in item_list.keys():
        item_list[item] = 0
        for row in data:
            #row : frozenset
            if item.issubset(row):
                # if the set is present in the transactions icrease its frequency by one
                item_list[item] += 1
    return item_list

In [85]:
# read data set
with open("groceries.csv") as data_set:
    reader = csv.reader(data_set, delimiter=',')
    for row in reader:
        data.append(frozenset(row))

In [86]:
# item list initialize

for row in data:
    for item in row:
        item_list[frozenset([item])] = 0

item_list = get_count(item_list,data)
item_list = support_check(item_list)

In [87]:
# generate next level until empty
level = 1
while True:
    level += 1

    # generate next level lists
    item_list = next_gen(item_list)
    item_list = get_count(item_list,data)
    item_list =support_check(item_list)
    if item_list == {}:
        break

# frequent set printing

In [88]:
for i in global_list.keys():
    print(list(i))

['citrus fruit']
['margarine']
['yogurt']
['coffee']
['tropical fruit']
['whole milk']
['cream cheese']
['pip fruit']
['long life bakery product']
['other vegetables']
['butter']
['rolls/buns']
['UHT-milk']
['bottled beer']
['white bread']
['bottled water']
['chocolate']
['curd']
['beef']
['frankfurter']
['soda']
['chicken']
['newspapers']
['fruit/vegetable juice']
['sugar']
['pastry']
['waffles']
['salty snack']
['root vegetables']
['canned beer']
['sausage']
['brown bread']
['shopping bags']
['hygiene articles']
['hamburger meat']
['napkins']
['whipped/sour cream']
['pork']
['berries']
['dessert']
['domestic eggs']
['frozen vegetables']
['onions']
['citrus fruit', 'whole milk']
['whole milk', 'yogurt']
['yogurt', 'other vegetables']
['rolls/buns', 'yogurt']
['whole milk', 'tropical fruit']
['other vegetables', 'tropical fruit']
['whole milk', 'other vegetables']
['rolls/buns', 'whole milk']
['whole milk', 'bottled water']
['soda', 'whole milk']
['pastry', 'whole milk']
['whole milk',

In [89]:
def giveconfidence(temp_lhs, temp_rhs):
    return global_list[frozenset(temp_rhs).union(frozenset(temp_lhs))] / (global_list[frozenset(temp_lhs)])

In [90]:
#function for Rules
def rule_gen(item_set):
    # rule generator given an item_set based on apriori principle
 
    item_set = list(item_set)
    if len(item_set) == 1:
        return None
    
    lhs = item_set
    rhs = []
    rules = [(lhs,rhs)]
    while len(rules) > 0:
        rule = rules[0]
        lhs = rule[0]
        rhs = rule[1]

        for i in lhs:
            temp_lhs,temp_rhs = lhs,rhs
            # remove one from lhs and add it to rhs
            temp_lhs.remove(i)
            temp_rhs.append(i)
            if temp_lhs == []:
                continue
            if giveconfidence(temp_lhs, temp_rhs) > MIN_CONF:
                # if conf is greater than given value add to the total list
                global_rule_list.append((temp_lhs,temp_rhs))
                print("{} -> {}".format(temp_lhs,temp_rhs))
                if len(temp_lhs) > 1:
                    rules.append((temp_lhs,temp_rhs))

        rules.remove(rule)

In [91]:
for item_set in global_list.keys():
    item_list = list(item_set)
    rule_gen(item_set)

['yogurt'] -> ['whole milk']
['tropical fruit'] -> ['whole milk']
['tropical fruit'] -> ['other vegetables']
['other vegetables'] -> ['whole milk']
['bottled water'] -> ['whole milk']
['root vegetables'] -> ['whole milk']


# Closed and Maximal

In [92]:
#initialized with true
maximal = {k:True for k in global_list.keys()}
closed = {k:True for k in global_list.keys()}

In [76]:
similar_pairs = []
for i in global_list.keys():
    for j in global_list.keys():
        if i != j:
            if i.issubset(j):
                # if its superset exists in frequent item set then it is not maximal
                maximal[i] = False
            if i.issubset(j) and global_list[i] == global_list[j]:
                similar_pairs.append((i,j)) # i = X,j = X'
                # if its superset has frequency equal to its own then it is not closed
                closed[i] = False

In [77]:
# all closed items
print({k:v for (k,v) in closed.items() if v == True})

{frozenset({'citrus fruit'}): True, frozenset({'margarine'}): True, frozenset({'yogurt'}): True, frozenset({'coffee'}): True, frozenset({'tropical fruit'}): True, frozenset({'whole milk'}): True, frozenset({'cream cheese'}): True, frozenset({'pip fruit'}): True, frozenset({'long life bakery product'}): True, frozenset({'other vegetables'}): True, frozenset({'butter'}): True, frozenset({'rolls/buns'}): True, frozenset({'UHT-milk'}): True, frozenset({'bottled beer'}): True, frozenset({'white bread'}): True, frozenset({'bottled water'}): True, frozenset({'chocolate'}): True, frozenset({'curd'}): True, frozenset({'beef'}): True, frozenset({'frankfurter'}): True, frozenset({'soda'}): True, frozenset({'chicken'}): True, frozenset({'newspapers'}): True, frozenset({'fruit/vegetable juice'}): True, frozenset({'sugar'}): True, frozenset({'pastry'}): True, frozenset({'waffles'}): True, frozenset({'salty snack'}): True, frozenset({'root vegetables'}): True, frozenset({'canned beer'}): True, frozen

In [78]:
# all maximal items
print([list(k) for (k,v) in maximal.items() if v == True])

[['margarine'], ['coffee'], ['cream cheese'], ['pip fruit'], ['long life bakery product'], ['butter'], ['UHT-milk'], ['bottled beer'], ['white bread'], ['chocolate'], ['curd'], ['beef'], ['frankfurter'], ['chicken'], ['newspapers'], ['fruit/vegetable juice'], ['sugar'], ['waffles'], ['salty snack'], ['canned beer'], ['brown bread'], ['shopping bags'], ['hygiene articles'], ['hamburger meat'], ['napkins'], ['pork'], ['berries'], ['dessert'], ['domestic eggs'], ['frozen vegetables'], ['onions'], ['citrus fruit', 'whole milk'], ['whole milk', 'yogurt'], ['yogurt', 'other vegetables'], ['rolls/buns', 'yogurt'], ['whole milk', 'tropical fruit'], ['other vegetables', 'tropical fruit'], ['whole milk', 'other vegetables'], ['rolls/buns', 'whole milk'], ['whole milk', 'bottled water'], ['soda', 'whole milk'], ['pastry', 'whole milk'], ['whole milk', 'root vegetables'], ['whipped/sour cream', 'whole milk'], ['rolls/buns', 'other vegetables'], ['soda', 'other vegetables'], ['root vegetables', 'ot

# Redundant Rules

In [79]:
# find redundant rules
redundant_rules = []
for pair1 in similar_pairs:
    for pair2 in similar_pairs:
        if pair1 != pair2 :
            lhs = frozenset(pair1[1])
            rhs = frozenset(pair2[1])
            conf = global_list[lhs.union(rhs)] / global_list[lhs]
            if conf > MIN_CONF:
                redundant_rules.append(pair1[0],pair2[0])

In [80]:
print(redundant_rules)

[]
